In [1]:
import pandas as pd

# Load the CSV file
csv_file = 'clubhouse24hrs_sata_data.csv'
df = pd.read_csv(csv_file)

# Print the first few rows
print("Head of the file:")
print(df.head())

# Print the last few rows
print("\nTail of the file:")
print(df.tail())


Head of the file:
   week  seconds  prn  elevation
0   267   119986    5      52.98
1   267   119986    5      52.98
2   267   119987    5      52.97
3   267   119988    5      52.97
4   267   119989    5      52.96

Tail of the file:
       week  seconds  prn  elevation
86404   267   206384   11      24.00
86405   267   206385   11      24.00
86406   267   206386   11      23.99
86407   267   206387   11      23.99
86408   267   206388   11      23.99


In [2]:
# Drop all columns except the specified ones
columns_to_keep = ['week', 'seconds', 'prn', 'elevation']
df = df[columns_to_keep]
df = df[df['seconds'] >= 0]


In [3]:
from datetime import datetime, timedelta

# Define the GPS epoch
GPS_EPOCH = datetime(1980, 1, 6)

def gps_to_utc(week, seconds):
    # Calculate the GPS time
    gps_time = GPS_EPOCH + timedelta(weeks=week, seconds=seconds)
    return gps_time

# Apply the conversion to the DataFrame
df['UTC Time'] = df.apply(lambda row: gps_to_utc(row['week'], row['seconds']), axis=1)

# Convert 'UTC Time' to a 24-hour time format
df['24-hour Time'] = df['UTC Time'].dt.strftime('%H:%M:%S')

# Print the first few rows to verify
print("Head of the file with 24-hour time format:")
print(df.head())


Head of the file with 24-hour time format:
   week  seconds  prn  elevation            UTC Time 24-hour Time
0   267   119986    5      52.98 1985-02-18 09:19:46     09:19:46
1   267   119986    5      52.98 1985-02-18 09:19:46     09:19:46
2   267   119987    5      52.97 1985-02-18 09:19:47     09:19:47
3   267   119988    5      52.97 1985-02-18 09:19:48     09:19:48
4   267   119989    5      52.96 1985-02-18 09:19:49     09:19:49


In [4]:
import plotly.io as pio
pio.renderers.default = "iframe"
# Ask user to input a specific PRN
import plotly.express as px
import plotly.graph_objects as go
# Plot d/u values against 24-hour time for all PRNs together
fig = go.Figure()

# Add traces for each PRN
for prn in df['prn'].unique():
    group = df[df['prn'] == prn]
    fig.add_trace(go.Scatter(x=group['24-hour Time'], y=group['elevation'], mode='lines', name=f'PRN {prn}'))

# Update layout
fig.update_layout(
    title='d/u values for all PRNs',
    xaxis_title='Time (24-hour format)',
    yaxis_title='Elevation',
    xaxis_tickformat='%H:%M:%S'
)

# Show the figure
fig.show()


In [5]:
grouped = df.groupby('prn')

# Print the head and tail of each group
for prn, group in grouped:
    print(f"\nPRN: {prn}")
    print("Head of the group:")
    print(group.head())
    print("\nTail of the group:")
    print(group.tail())


PRN: 2
Head of the group:
       week  seconds  prn  elevation            UTC Time 24-hour Time
60948   267   180929    2      25.31 1985-02-19 02:15:29     02:15:29
60959   267   180940    2      25.24 1985-02-19 02:15:40     02:15:40
60960   267   180941    2      25.23 1985-02-19 02:15:41     02:15:41
60963   267   180944    2      25.21 1985-02-19 02:15:44     02:15:44
60964   267   180945    2      25.21 1985-02-19 02:15:45     02:15:45

Tail of the group:
       week  seconds  prn  elevation            UTC Time 24-hour Time
62238   267   182219    2      17.26 1985-02-19 02:36:59     02:36:59
62239   267   182220    2      17.25 1985-02-19 02:37:00     02:37:00
62240   267   182221    2      17.24 1985-02-19 02:37:01     02:37:01
62241   267   182222    2      17.24 1985-02-19 02:37:02     02:37:02
62242   267   182223    2      17.23 1985-02-19 02:37:03     02:37:03

PRN: 5
Head of the group:
   week  seconds  prn  elevation            UTC Time 24-hour Time
0   267   119986    

In [6]:
# Ask user to input a specific PRN
import plotly.express as px
import plotly.graph_objects as go
specific_prn = int(input("Enter a PRN to plot (e.g., 32): "))

# Plot d/u values against 24-hour time for the specific PRN
if specific_prn in grouped.groups:
    group = grouped.get_group(specific_prn)
    fig = px.line(group, x='24-hour Time', y='elevation', title=f'd/u values for PRN {specific_prn}')
    fig.update_layout(
        xaxis_title='Time (24-hour format)',
        yaxis_title='Elevation',
        xaxis_tickformat='%H:%M:%S'
    )
    fig.show()
else:
    print(f"PRN {specific_prn} not found in the dataset.")


Enter a PRN to plot (e.g., 32):  6


PRN 6 not found in the dataset.
